In [2]:
import nibabel as nl
import numpy as np
import pandas as pd
import os, shutil

In [3]:
df = pd.read_csv('df_with_schaefer_brainnetome_zALFF_CorticalRemoved.csv')
df.head()

,Unnamed: 0,Subject_ID,imaging waves,snr_total,zALFF_17Networks_RH_TempPar_10,zALFF_17Networks_RH_TempPar_9,zALFF_17Networks_RH_TempPar_8,zALFF_17Networks_RH_TempPar_7,zALFF_17Networks_RH_TempPar_6,zALFF_17Networks_RH_TempPar_5,...,summary_wm_mean,summary_wm_median,summary_wm_n,summary_wm_p05,summary_wm_p95,summary_wm_stdv,tpm_overlap_csf,tpm_overlap_gm,tpm_overlap_wm,wm2max
0,0,sub-0015,ses-wave3,11.837327,-0.412583,-0.613279,-0.316116,-0.002759,-0.040997,-0.371437,...,995.745911,1000.011963,315498.0,901.995431,1071.995032,51.632389,0.158227,0.463963,0.496865,0.348531
1,1,sub-0085,ses-wave2,11.131926,-0.372482,-0.197315,-0.226332,-0.183162,-0.236034,-0.053904,...,997.301331,1000.002930,263420.0,909.174200,1072.202417,51.231632,0.124368,0.418898,0.468719,0.624520
2,2,sub-0134,ses-wave3,10.562882,0.038523,-0.133351,-0.071341,0.030456,0.068658,-0.139336,...,999.545776,1000.019165,320010.0,905.605655,1090.527765,56.056854,0.145637,0.456474,0.484460,0.418354
3,3,sub-0152,ses-wave2,12.745413,-0.227420,-0.604107,-0.255079,-0.216819,-0.174185,-0.474216,...,998.156128,1000.008911,256839.0,915.794702,1072.669592,47.963997,0.165062,0.447994,0.489054,0.641355
4,4,sub-0260,ses-wave2,11.458236,-0.096345,-0.251739,-0.037423,-0.211528,-0.047426,-0.089775,...,997.054321,1000.033447,309014.0,896.412515,1085.178082,56.937580,0.167600,0.473556,0.492470,0.648387


In [4]:
df = df.drop('Unnamed: 0', axis=1)

In [5]:
df.head()

,Subject_ID,imaging waves,snr_total,zALFF_17Networks_RH_TempPar_10,zALFF_17Networks_RH_TempPar_9,zALFF_17Networks_RH_TempPar_8,zALFF_17Networks_RH_TempPar_7,zALFF_17Networks_RH_TempPar_6,zALFF_17Networks_RH_TempPar_5,zALFF_17Networks_RH_TempPar_4,...,summary_wm_mean,summary_wm_median,summary_wm_n,summary_wm_p05,summary_wm_p95,summary_wm_stdv,tpm_overlap_csf,tpm_overlap_gm,tpm_overlap_wm,wm2max
0,sub-0015,ses-wave3,11.837327,-0.412583,-0.613279,-0.316116,-0.002759,-0.040997,-0.371437,0.005503,...,995.745911,1000.011963,315498.0,901.995431,1071.995032,51.632389,0.158227,0.463963,0.496865,0.348531
1,sub-0085,ses-wave2,11.131926,-0.372482,-0.197315,-0.226332,-0.183162,-0.236034,-0.053904,-0.022926,...,997.301331,1000.002930,263420.0,909.174200,1072.202417,51.231632,0.124368,0.418898,0.468719,0.624520
2,sub-0134,ses-wave3,10.562882,0.038523,-0.133351,-0.071341,0.030456,0.068658,-0.139336,0.287498,...,999.545776,1000.019165,320010.0,905.605655,1090.527765,56.056854,0.145637,0.456474,0.484460,0.418354
3,sub-0152,ses-wave2,12.745413,-0.227420,-0.604107,-0.255079,-0.216819,-0.174185,-0.474216,-0.116556,...,998.156128,1000.008911,256839.0,915.794702,1072.669592,47.963997,0.165062,0.447994,0.489054,0.641355
4,sub-0260,ses-wave2,11.458236,-0.096345,-0.251739,-0.037423,-0.211528,-0.047426,-0.089775,0.348758,...,997.054321,1000.033447,309014.0,896.412515,1085.178082,56.937580,0.167600,0.473556,0.492470,0.648387


## adding zfALFF of BN to dataframe

### list of zfALFF (BN)

In [6]:
# opening the text file
lis = []
with open('zfALFF_BN_meants.txt','r') as ALFFout:
   
    # reading each line    
    for line in ALFFout:
   
        # reading each word        
        for ALFF in line.split():
            lis.append(float(ALFF))
            # displaying the words           
            #print(ALFF) 

In [7]:
lis

[0.1080745791,
 0.09111422183,
 0.1414322306,
 0.2969189588,
 0.2385657844,
 0.1914342398,
 0.1036438624,
 0.1696590097,
 0.07575548358,
 0.1178842079,
 0.2280413993,
 0.2512160444,
 0.3007383122,
 0.187229934,
 0.2493946435,
 0.2199514643,
 0.04106421114,
 0.1465615472,
 0.1677826093,
 0.3327527494,
 0.1070803731,
 0.3719887187,
 0.2984164602,
 0.4246040854,
 0.04010523718,
 0.2076921899,
 0.270432302,
 0.2585626295,
 0.09068264557,
 0.2217349119,
 0.05054439674,
 0.2007450639,
 0.2938676928,
 0.3462946653,
 0.1461439206,
 0.3455818373,
 -0.02287588231,
 -0.0184984366,
 0.08880699162,
 0.02927972421,
 0.1458906129,
 -0.06903918328,
 -0.1863862365,
 -0.08332431569,
 0.1125790312,
 0.05233214667,
 0.3368174145,
 0.2684538408,
 -0.05730722504,
 -0.09711837309,
 -0.02536883022,
 0.1040136365,
 0.2012496644,
 0.001860211283,
 -0.101788922,
 0.1675902899,
 -0.1034594429,
 0.03674123303,
 0.09621663916,
 -0.05200419381,
 -0.01991449341,
 0.09826917322,
 0.225931288,
 0.3521083782,
 0.1353833

### list of region names in BN Atlas (246parc)

In [8]:
token = open('BN_Atlas_246_LUT.txt','r')
linestoken=token.readlines()
tokens_column_number = 1
parname=[]
for x in linestoken:
    parname.append(x.split()[tokens_column_number])
token.close()
print(parname)

['A8m_L', 'A8m_R', 'A8dl_L', 'A8dl_R', 'A9l_L', 'A9l_R', 'A6dl_L', 'A6dl_R', 'A6m_L', 'A6m_R', 'A9m_L', 'A9m_R', 'A10m_L', 'A10m_R', 'A9/46d_L', 'A9/46d_R', 'IFJ_L', 'IFJ_R', 'A46_L', 'A46_R', 'A9/46v_L', 'A9/46v_R', 'A8vl_L', 'A8vl_R', 'A6vl_L', 'A6vl_R', 'A10l_L', 'A10l_R', 'A44d_L', 'A44d_R', 'IFS_L', 'IFS_R', 'A45c_L', 'A45c_R', 'A45r_L', 'A45r_R', 'A44op_L', 'A44op_R', 'A44v_L', 'A44v_R', 'A14m_L', 'A14m_R', 'A12/47o_L', 'A12/47o_R', 'A11l_L', 'A11l_R', 'A11m_L', 'A11m_R', 'A13_L', 'A13_R', 'A12/47l_L', 'A12/47l_R', 'A4hf_L', 'A4hf_R', 'A6cdl_L', 'A6cdl_R', 'A4ul_L', 'A4ul_R', 'A4t_L', 'A4t_R', 'A4tl_L', 'A4tl_R', 'A6cvl_L', 'A6cvl_R', 'A1/2/3ll_L', 'A1/2/3ll_R', 'A4ll_L', 'A4ll_R', 'A38m_L', 'A38m_R', 'A41/42_L', 'A41/42_R', 'TE1.0/TE1.2_L', 'TE1.0/TE1.2_R', 'A22c_L', 'A22c_R', 'A38l_L', 'A38l_R', 'A22r_L', 'A22r_R', 'A21c_L', 'A21c_R', 'A21r_L', 'A21r_R', 'A37dl_L', 'A37dl_R', 'aSTS_L', 'aSTS_R', 'A20iv_L', 'A20iv_R', 'A37elv_L', 'A37elv_R', 'A20r_L', 'A20r_R', 'A20il_L', 'A20il

In [9]:
len(parname)

246

### merging regions and their values with dataframe

In [10]:
test = df.set_index('Subject_ID')
inds = test.index
test.head()

,imaging waves,snr_total,zALFF_17Networks_RH_TempPar_10,zALFF_17Networks_RH_TempPar_9,zALFF_17Networks_RH_TempPar_8,zALFF_17Networks_RH_TempPar_7,zALFF_17Networks_RH_TempPar_6,zALFF_17Networks_RH_TempPar_5,zALFF_17Networks_RH_TempPar_4,zALFF_17Networks_RH_TempPar_3,...,summary_wm_mean,summary_wm_median,summary_wm_n,summary_wm_p05,summary_wm_p95,summary_wm_stdv,tpm_overlap_csf,tpm_overlap_gm,tpm_overlap_wm,wm2max
Subject_ID,,,,,,,,,,,,,,,,,,,,,
sub-0015,ses-wave3,11.837327,-0.412583,-0.613279,-0.316116,-0.002759,-0.040997,-0.371437,0.005503,-0.134194,...,995.745911,1000.011963,315498.0,901.995431,1071.995032,51.632389,0.158227,0.463963,0.496865,0.348531
sub-0085,ses-wave2,11.131926,-0.372482,-0.197315,-0.226332,-0.183162,-0.236034,-0.053904,-0.022926,-0.001857,...,997.301331,1000.002930,263420.0,909.174200,1072.202417,51.231632,0.124368,0.418898,0.468719,0.624520
sub-0134,ses-wave3,10.562882,0.038523,-0.133351,-0.071341,0.030456,0.068658,-0.139336,0.287498,-0.010053,...,999.545776,1000.019165,320010.0,905.605655,1090.527765,56.056854,0.145637,0.456474,0.484460,0.418354
sub-0152,ses-wave2,12.745413,-0.227420,-0.604107,-0.255079,-0.216819,-0.174185,-0.474216,-0.116556,-0.388770,...,998.156128,1000.008911,256839.0,915.794702,1072.669592,47.963997,0.165062,0.447994,0.489054,0.641355
sub-0260,ses-wave2,11.458236,-0.096345,-0.251739,-0.037423,-0.211528,-0.047426,-0.089775,0.348758,0.148863,...,997.054321,1000.033447,309014.0,896.412515,1085.178082,56.937580,0.167600,0.473556,0.492470,0.648387


In [11]:
df = pd.DataFrame()
for j, ind in enumerate(inds):
    sub = test.loc[[ind]]
    for i, pars in enumerate(parname):
        col_name = 'zfALFF_'+str(pars)
        if col_name not in sub:
            sub.insert(2, col_name, np.nan, True)
        sub[col_name] = lis[i+j*246]
    df = pd.concat([df, sub])

In [12]:
df.head()

,imaging waves,snr_total,zfALFF_lPFtha_R,zfALFF_lPFtha_L,zfALFF_cTtha_R,zfALFF_cTtha_L,zfALFF_Otha_R,zfALFF_Otha_L,zfALFF_PPtha_R,zfALFF_PPtha_L,...,summary_wm_mean,summary_wm_median,summary_wm_n,summary_wm_p05,summary_wm_p95,summary_wm_stdv,tpm_overlap_csf,tpm_overlap_gm,tpm_overlap_wm,wm2max
Subject_ID,,,,,,,,,,,,,,,,,,,,,
sub-0015,ses-wave3,11.837327,-0.760500,-0.929277,-0.564227,-0.743303,-0.649522,-0.701746,-0.505629,-0.701899,...,995.745911,1000.011963,315498.0,901.995431,1071.995032,51.632389,0.158227,0.463963,0.496865,0.348531
sub-0085,ses-wave2,11.131926,-0.234355,-0.397031,-0.182309,-0.381863,-0.308232,-0.449357,-0.218163,-0.394808,...,997.301331,1000.002930,263420.0,909.174200,1072.202417,51.231632,0.124368,0.418898,0.468719,0.624520
sub-0134,ses-wave3,10.562882,-1.012589,-0.840667,-0.760328,-0.901478,-0.891105,-0.487365,-0.727667,-0.935409,...,999.545776,1000.019165,320010.0,905.605655,1090.527765,56.056854,0.145637,0.456474,0.484460,0.418354
sub-0152,ses-wave2,12.745413,-0.682773,-0.294288,-0.123090,-0.634390,-0.701496,-0.597999,-0.759437,-0.702566,...,998.156128,1000.008911,256839.0,915.794702,1072.669592,47.963997,0.165062,0.447994,0.489054,0.641355
sub-0260,ses-wave2,11.458236,-0.814174,-0.863769,-0.709799,-0.617213,-0.775317,-0.475704,-0.541087,-0.716565,...,997.054321,1000.033447,309014.0,896.412515,1085.178082,56.937580,0.167600,0.473556,0.492470,0.648387


In [13]:
df.to_csv("df_with_zfALFF_brainnetome.csv")

## adding zALFF of Schaefer to the dataframe

### list of zALFF (Schaefer)

In [14]:
df = pd.read_csv('df_with_zfALFF_brainnetome.csv')
df.head()

,Subject_ID,imaging waves,snr_total,zfALFF_lPFtha_R,zfALFF_lPFtha_L,zfALFF_cTtha_R,zfALFF_cTtha_L,zfALFF_Otha_R,zfALFF_Otha_L,zfALFF_PPtha_R,...,summary_wm_mean,summary_wm_median,summary_wm_n,summary_wm_p05,summary_wm_p95,summary_wm_stdv,tpm_overlap_csf,tpm_overlap_gm,tpm_overlap_wm,wm2max
0,sub-0015,ses-wave3,11.837327,-0.760500,-0.929277,-0.564227,-0.743303,-0.649522,-0.701746,-0.505629,...,995.745911,1000.011963,315498.0,901.995431,1071.995032,51.632389,0.158227,0.463963,0.496865,0.348531
1,sub-0085,ses-wave2,11.131926,-0.234355,-0.397031,-0.182309,-0.381863,-0.308232,-0.449357,-0.218163,...,997.301331,1000.002930,263420.0,909.174200,1072.202417,51.231632,0.124368,0.418898,0.468719,0.624520
2,sub-0134,ses-wave3,10.562882,-1.012589,-0.840667,-0.760328,-0.901478,-0.891105,-0.487365,-0.727667,...,999.545776,1000.019165,320010.0,905.605655,1090.527765,56.056854,0.145637,0.456474,0.484460,0.418354
3,sub-0152,ses-wave2,12.745413,-0.682773,-0.294288,-0.123090,-0.634390,-0.701496,-0.597999,-0.759437,...,998.156128,1000.008911,256839.0,915.794702,1072.669592,47.963997,0.165062,0.447994,0.489054,0.641355
4,sub-0260,ses-wave2,11.458236,-0.814174,-0.863769,-0.709799,-0.617213,-0.775317,-0.475704,-0.541087,...,997.054321,1000.033447,309014.0,896.412515,1085.178082,56.937580,0.167600,0.473556,0.492470,0.648387


In [15]:
# opening the text file
lis = []
with open('zfALFF_Schaefer_meants.txt','r') as ALFFout:
   
    # reading each line    
    for line in ALFFout:
   
        # reading each word        
        for ALFF in line.split():
            lis.append(float(ALFF))
            # displaying the words           
            #print(ALFF) 

In [16]:
lis

[0.2569220347,
 0.2623580252,
 0.3323617167,
 0.211860752,
 0.09118900392,
 0.1358083786,
 0.008971374109,
 0.3410795408,
 0.65090202,
 0.1839136687,
 0.3657957336,
 0.2994070099,
 0.09288248187,
 0.3874732696,
 0.4670590886,
 0.07325296526,
 0.2262455525,
 0.5261225409,
 0.3783302606,
 0.1951385463,
 0.7247192727,
 -0.1710421294,
 0.4685634524,
 0.04776694394,
 -0.1875401893,
 0.2730971623,
 0.1388934026,
 0.1714870908,
 0.0158107117,
 -0.1392611289,
 -0.06279446034,
 0.01477146868,
 -0.04741069953,
 0.01185245376,
 0.1754041196,
 -0.02432966064,
 -0.1928151977,
 0.2344227921,
 0.1274299136,
 0.1546724535,
 -0.06138772256,
 -0.02418392756,
 -0.0458578523,
 -0.3402351155,
 -0.06288016647,
 -0.2340700921,
 -0.2936817964,
 -0.444600056,
 -0.106677546,
 -0.2881684566,
 0.0008014887571,
 0.07967592867,
 0.02924924272,
 0.08095279937,
 0.1758810183,
 0.2167618773,
 0.4118252473,
 0.1999351073,
 0.295592018,
 0.1311350067,
 0.04874283446,
 0.4317923176,
 0.4512801764,
 0.1262164277,
 0.35789

### list of region names in Schaefer Atlas (400 parc-17 net)

In [17]:
token = open('Schaefer2018_400Parcels_17Networks_order.txt','r')
linestoken=token.readlines()
tokens_column_number = 1
parname=[]
for x in linestoken:
    parname.append(x.split()[tokens_column_number])
token.close()
print(parname)

['17Networks_LH_VisCent_ExStr_1', '17Networks_LH_VisCent_ExStr_2', '17Networks_LH_VisCent_ExStr_3', '17Networks_LH_VisCent_ExStr_4', '17Networks_LH_VisCent_ExStr_5', '17Networks_LH_VisCent_ExStr_6', '17Networks_LH_VisCent_Striate_1', '17Networks_LH_VisCent_ExStr_7', '17Networks_LH_VisCent_ExStr_8', '17Networks_LH_VisCent_ExStr_9', '17Networks_LH_VisCent_ExStr_10', '17Networks_LH_VisCent_ExStr_11', '17Networks_LH_VisPeri_ExStrInf_1', '17Networks_LH_VisPeri_ExStrInf_2', '17Networks_LH_VisPeri_ExStrInf_3', '17Networks_LH_VisPeri_ExStrInf_4', '17Networks_LH_VisPeri_ExStrInf_5', '17Networks_LH_VisPeri_StriCal_1', '17Networks_LH_VisPeri_StriCal_2', '17Networks_LH_VisPeri_ExStrSup_1', '17Networks_LH_VisPeri_ExStrSup_2', '17Networks_LH_VisPeri_ExStrSup_3', '17Networks_LH_VisPeri_ExStrSup_4', '17Networks_LH_VisPeri_ExStrSup_5', '17Networks_LH_SomMotA_1', '17Networks_LH_SomMotA_2', '17Networks_LH_SomMotA_3', '17Networks_LH_SomMotA_4', '17Networks_LH_SomMotA_5', '17Networks_LH_SomMotA_6', '17Netw

In [18]:
#len(parname)
47200/118

400.0

### merging regions and their values with dataframe

In [19]:
df = pd.read_csv('df_with_zfALFF_brainnetome.csv')
test = df.set_index('Subject_ID')
inds = test.index
#inds
df.head()

,Subject_ID,imaging waves,snr_total,zfALFF_lPFtha_R,zfALFF_lPFtha_L,zfALFF_cTtha_R,zfALFF_cTtha_L,zfALFF_Otha_R,zfALFF_Otha_L,zfALFF_PPtha_R,...,summary_wm_mean,summary_wm_median,summary_wm_n,summary_wm_p05,summary_wm_p95,summary_wm_stdv,tpm_overlap_csf,tpm_overlap_gm,tpm_overlap_wm,wm2max
0,sub-0015,ses-wave3,11.837327,-0.760500,-0.929277,-0.564227,-0.743303,-0.649522,-0.701746,-0.505629,...,995.745911,1000.011963,315498.0,901.995431,1071.995032,51.632389,0.158227,0.463963,0.496865,0.348531
1,sub-0085,ses-wave2,11.131926,-0.234355,-0.397031,-0.182309,-0.381863,-0.308232,-0.449357,-0.218163,...,997.301331,1000.002930,263420.0,909.174200,1072.202417,51.231632,0.124368,0.418898,0.468719,0.624520
2,sub-0134,ses-wave3,10.562882,-1.012589,-0.840667,-0.760328,-0.901478,-0.891105,-0.487365,-0.727667,...,999.545776,1000.019165,320010.0,905.605655,1090.527765,56.056854,0.145637,0.456474,0.484460,0.418354
3,sub-0152,ses-wave2,12.745413,-0.682773,-0.294288,-0.123090,-0.634390,-0.701496,-0.597999,-0.759437,...,998.156128,1000.008911,256839.0,915.794702,1072.669592,47.963997,0.165062,0.447994,0.489054,0.641355
4,sub-0260,ses-wave2,11.458236,-0.814174,-0.863769,-0.709799,-0.617213,-0.775317,-0.475704,-0.541087,...,997.054321,1000.033447,309014.0,896.412515,1085.178082,56.937580,0.167600,0.473556,0.492470,0.648387


In [20]:
df = pd.DataFrame()
for j, ind in enumerate(inds):
    sub = test.loc[[ind]]
    for i, pars in enumerate(parname):
        col_name = 'zfALFF_'+str(pars)
        if col_name not in sub:
            sub.insert(2, col_name, np.nan, True)
        sub[col_name] = lis[i+j*400]
    df = pd.concat([df, sub])

In [21]:
df.head()

,imaging waves,snr_total,zfALFF_17Networks_RH_TempPar_10,zfALFF_17Networks_RH_TempPar_9,zfALFF_17Networks_RH_TempPar_8,zfALFF_17Networks_RH_TempPar_7,zfALFF_17Networks_RH_TempPar_6,zfALFF_17Networks_RH_TempPar_5,zfALFF_17Networks_RH_TempPar_4,zfALFF_17Networks_RH_TempPar_3,...,summary_wm_mean,summary_wm_median,summary_wm_n,summary_wm_p05,summary_wm_p95,summary_wm_stdv,tpm_overlap_csf,tpm_overlap_gm,tpm_overlap_wm,wm2max
Subject_ID,,,,,,,,,,,,,,,,,,,,,
sub-0015,ses-wave3,11.837327,0.101760,-0.155537,0.188868,0.417086,0.480759,0.032914,0.667444,0.261053,...,995.745911,1000.011963,315498.0,901.995431,1071.995032,51.632389,0.158227,0.463963,0.496865,0.348531
sub-0085,ses-wave2,11.131926,0.109652,0.450457,0.391265,0.398096,0.152959,-0.058422,-0.237084,-0.262974,...,997.301331,1000.002930,263420.0,909.174200,1072.202417,51.231632,0.124368,0.418898,0.468719,0.624520
sub-0134,ses-wave3,10.562882,0.194081,-0.011667,0.341790,0.356530,0.254256,0.079741,0.349756,0.128864,...,999.545776,1000.019165,320010.0,905.605655,1090.527765,56.056854,0.145637,0.456474,0.484460,0.418354
sub-0152,ses-wave2,12.745413,-0.077667,-0.631299,-0.227984,-0.193782,-0.469331,-1.500759,-0.408363,-0.566876,...,998.156128,1000.008911,256839.0,915.794702,1072.669592,47.963997,0.165062,0.447994,0.489054,0.641355
sub-0260,ses-wave2,11.458236,0.226929,0.336683,0.265665,0.425873,0.271967,0.156014,0.528694,-0.141647,...,997.054321,1000.033447,309014.0,896.412515,1085.178082,56.937580,0.167600,0.473556,0.492470,0.648387


In [22]:
df.to_csv("df_with_zfALFF_brainnetome_Schaefer.csv")

### removing cortical regions of BN 

In [23]:
df = pd.read_csv("df_with_zfALFF_brainnetome_Schaefer.csv")
df.head()

,Subject_ID,imaging waves,snr_total,zfALFF_17Networks_RH_TempPar_10,zfALFF_17Networks_RH_TempPar_9,zfALFF_17Networks_RH_TempPar_8,zfALFF_17Networks_RH_TempPar_7,zfALFF_17Networks_RH_TempPar_6,zfALFF_17Networks_RH_TempPar_5,zfALFF_17Networks_RH_TempPar_4,...,summary_wm_mean,summary_wm_median,summary_wm_n,summary_wm_p05,summary_wm_p95,summary_wm_stdv,tpm_overlap_csf,tpm_overlap_gm,tpm_overlap_wm,wm2max
0,sub-0015,ses-wave3,11.837327,0.101760,-0.155537,0.188868,0.417086,0.480759,0.032914,0.667444,...,995.745911,1000.011963,315498.0,901.995431,1071.995032,51.632389,0.158227,0.463963,0.496865,0.348531
1,sub-0085,ses-wave2,11.131926,0.109652,0.450457,0.391265,0.398096,0.152959,-0.058422,-0.237084,...,997.301331,1000.002930,263420.0,909.174200,1072.202417,51.231632,0.124368,0.418898,0.468719,0.624520
2,sub-0134,ses-wave3,10.562882,0.194081,-0.011667,0.341790,0.356530,0.254256,0.079741,0.349756,...,999.545776,1000.019165,320010.0,905.605655,1090.527765,56.056854,0.145637,0.456474,0.484460,0.418354
3,sub-0152,ses-wave2,12.745413,-0.077667,-0.631299,-0.227984,-0.193782,-0.469331,-1.500759,-0.408363,...,998.156128,1000.008911,256839.0,915.794702,1072.669592,47.963997,0.165062,0.447994,0.489054,0.641355
4,sub-0260,ses-wave2,11.458236,0.226929,0.336683,0.265665,0.425873,0.271967,0.156014,0.528694,...,997.054321,1000.033447,309014.0,896.412515,1085.178082,56.937580,0.167600,0.473556,0.492470,0.648387


In [25]:
df.columns.get_loc('zfALFF_lsOccG_R')

439

In [26]:
df.columns.get_loc('zfALFF_A8m_L')

648

In [27]:
df.drop(df.columns[439:648], axis = 1, inplace = True)
  
df.head()

,Subject_ID,imaging waves,snr_total,zfALFF_17Networks_RH_TempPar_10,zfALFF_17Networks_RH_TempPar_9,zfALFF_17Networks_RH_TempPar_8,zfALFF_17Networks_RH_TempPar_7,zfALFF_17Networks_RH_TempPar_6,zfALFF_17Networks_RH_TempPar_5,zfALFF_17Networks_RH_TempPar_4,...,summary_wm_mean,summary_wm_median,summary_wm_n,summary_wm_p05,summary_wm_p95,summary_wm_stdv,tpm_overlap_csf,tpm_overlap_gm,tpm_overlap_wm,wm2max
0,sub-0015,ses-wave3,11.837327,0.101760,-0.155537,0.188868,0.417086,0.480759,0.032914,0.667444,...,995.745911,1000.011963,315498.0,901.995431,1071.995032,51.632389,0.158227,0.463963,0.496865,0.348531
1,sub-0085,ses-wave2,11.131926,0.109652,0.450457,0.391265,0.398096,0.152959,-0.058422,-0.237084,...,997.301331,1000.002930,263420.0,909.174200,1072.202417,51.231632,0.124368,0.418898,0.468719,0.624520
2,sub-0134,ses-wave3,10.562882,0.194081,-0.011667,0.341790,0.356530,0.254256,0.079741,0.349756,...,999.545776,1000.019165,320010.0,905.605655,1090.527765,56.056854,0.145637,0.456474,0.484460,0.418354
3,sub-0152,ses-wave2,12.745413,-0.077667,-0.631299,-0.227984,-0.193782,-0.469331,-1.500759,-0.408363,...,998.156128,1000.008911,256839.0,915.794702,1072.669592,47.963997,0.165062,0.447994,0.489054,0.641355
4,sub-0260,ses-wave2,11.458236,0.226929,0.336683,0.265665,0.425873,0.271967,0.156014,0.528694,...,997.054321,1000.033447,309014.0,896.412515,1085.178082,56.937580,0.167600,0.473556,0.492470,0.648387


In [29]:
df.to_csv("df_with_schaefer_brainnetome_zfALFF_CorticalRemoved.csv")